In [ ]:
import numpy as np
from iminuit import Minuit

import clapy

import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
from math import sqrt,log
def fitfunc( t, Tc, r, GF):
    if t<(Tc-Tc*r):
        res = GF/Tc*(t+Tc*r)
    else:
        res =  GF
    return res 

def mypdfln( x,  mu,  std):
    #cpdef means generate both c function and python function
    norm = (sqrt(2*np.pi*std*std))
    ret = (-1*(x-mu)*(x-mu)/(2.*std*std))-log(norm)
    return ret




class Nowakowski_LH:  
    def __init__(self,data,t):
        self.data = data
        self.t = t
        self.ndata = len(data)
    
    def compute(self,  Tc, r, GF, s):
        #this line is a cast not a copy. Let cython knows mydata will spit out double
        mydata = self.data
        myt = self.t
        loglh = 0.
        lmu = 0.
        ler = 0.
        for i in range(self.ndata):
            lmu = fitfunc(myt[i],Tc,r,GF)
            loglh -= mypdfln(mydata[i],lmu,s)
        return loglh



In [ ]:
#gen data need clasim
'''
import clasim
dargs = {
        'samples': 5,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.5,
        'S': 0.3,
        'G2M': 0.2,
        'sCells' : 0.3,
        'sSamples' : 0.2
}
timep = [0.1,0.4,0.5,0.8,1.0,1.2]
ncells = dargs['nCells']
labeld_cells,times = np.array(clasim.run(seed=int(np.random.rand()*1000),mode=1,times=timep,**dargs)) 
'''

In [ ]:
times  = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 
                0.4, 0.4, 0.4, 0.4, 0.4, 
                0.5, 0.5, 0.5, 0.5, 0.5,
                0.8, 0.8, 0.8, 0.8, 0.8, 
                1.2, 1.2, 1.2, 1.2, 1.2]) 
ncells = 100
labeld_cells = np.array([30., 49., 34., 43., 36.,
                      77., 74., 70., 56., 68.,
                      86., 84., 76., 86., 89.,
                      82., 87., 96., 91., 95.,
                      95., 94., 95., 93., 91.])

In [ ]:
clapy.web_fit(times,labeld_cells,ncells*np.ones_like(times),'.','name')

In [ ]:

if np.size(ncells) !=  np.size(labeld_cells):
    ncells = np.ones_like(labeld_cells)*ncells

#fit nowakowski
lh = Nowakowski_LH(labeld_cells/ncells,times)
mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=1,s=0.1,\
                      error_r=0.1,error_GF=0.1,error_Tc=0.1,error_s=0.1,\
           limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_s=(0.0001,1),\
           errordef=0.5,print_level=2)
mi_old.migrad()
res = mi_old.values
args = {
        'GF': res['GF'],
        'G1': res['Tc']*((1-res['r'])*0.5),
        'S': res['Tc']*res['r'],
        'G2M': res['Tc']*((1-res['r'])*0.5)
}

lhn = clapy.asym_lh(labeld_cells,times,ncells)
mi = Minuit(lhn.compute, Tc=1.0, r=0.3,GF=1.0,fix_GF=False,sigma_sample=0.25,sigma_cell=0.3, \
           error_Tc=0.1,error_r=0.1,error_GF=0.1,error_sigma_sample=0.1,error_sigma_cell=0.1,\
           limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_sigma_sample=(0.00001,1),limit_sigma_cell=(0.00001,1),\
           errordef=0.5,print_level=2)
mi.migrad();

fit = dict()
for i in  mi.values:
    fit.update( {i : {'value' : mi.values[i], '2sigma' : 2*mi.errors[i]}})


plt.plot(times,labeld_cells/ncells,'.')

ttt = np.linspace(0,times.max(),1000)
plt.plot(ttt,clapy.cla_det_model(ttt,mode=1,**args),':',label="now. fit")


ttt2 = np.linspace(0.001,times.max(),100)

d = clapy.dist()
prob = np.zeros(len(ttt2))
for t_n,t in enumerate(ttt2):
    prob[t_n] = d.pmf_mean(np.mean(ncells),\
                           fit['Tc']['value'],fit['r']['value'],fit['GF']['value'],\
                           fit['sigma_cell']['value'],fit['sigma_sample']['value'],t)\
                / np.mean(ncells)

plt.plot(ttt2,prob,label='probabilistic model')